# Experimental testing of existing models on a sample of Early Modern Dutch from the VOC archives

In [1]:
# Defining sample from archive
example = "Mier sahit Inhamet, op ontfangst van dat „schrijvens, dato 29„e xb: te laten versoecken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden"

In [2]:
# Importing relevant packages and loading fill-mask pipelines for all models we will test

import pandas as pd
import spacy
from transformers import pipeline
p_rob = pipeline("fill-mask", model="pdelobelle/robbert-v2-dutch-base")
p_gys = pipeline("fill-mask", model="emanjavacas/GysBERT")
p_gys2 = pipeline("fill-mask", model="emanjavacas/GysBERT-v2")
p_xlm = pipeline("fill-mask", model="xlm-roberta-large")
p_bertje = pipeline("fill-mask", model="GroNLP/bert-base-dutch-cased")

Some weights of the model checkpoint at emanjavacas/GysBERT were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Part 1: POS tagging

In [3]:
# Get POS-tags for archival sample from small spaCy model 

nlp = spacy.load("nl_core_news_sm")
doc = nlp(example)
print(doc.text)
for token in doc:
    print(token.text, ';' , token.pos_, ';', token.dep_)

Mier sahit Inhamet, op ontfangst van dat „schrijvens, dato 29„e xb: te laten versoecken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden
Mier ; PROPN ; ROOT
sahit ; NOUN ; parataxis
Inhamet ; PROPN ; flat
, ; PUNCT ; punct
op ; ADP ; case
ontfangst ; NOUN ; appos
van ; ADP ; case
dat ; DET ; mark
„ ; SYM ; det
schrijvens ; NOUN ; flat
, ; PUNCT ; punct
dato ; ADV ; conj
29„e ; SYM ; conj
xb ; CCONJ ; amod
: ; PUNCT ; punct
te ; ADP ; mark
laten ; VERB ; nmod
versoecken ; VERB ; ccomp
, ; PUNCT ; punct
dat ; SCONJ ; mark
hij ; PRON ; nsubj
ten ; ADP ; case
Eersten ; X ; obl
ordre ; PROPN ; appos
geliefde ; ADJ ; xcomp
te ; ADP ; mark
stellen ; VERB ; ccomp
, ; PUNCT ; punct
aen ; VERB ; obl
wie ; PRON 

In [4]:
# Get POS-tags for archival sample from big spaCy model 

nlp = spacy.load("nl_core_news_lg")
doc = nlp(example)
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

Mier sahit Inhamet, op ontfangst van dat „schrijvens, dato 29„e xb: te laten versoecken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden
Mier PROPN ROOT
sahit NOUN appos
Inhamet PROPN ROOT
, PUNCT punct
op ADP case
ontfangst NOUN obl
van ADP case
dat SCONJ mark
„ PUNCT det
schrijvens NOUN obj
, PUNCT punct
dato ADV flat
29„e SYM flat
xb SYM punct
: PUNCT punct
te ADP mark
laten VERB nmod
versoecken NOUN ccomp
, PUNCT punct
dat SCONJ mark
hij PRON nsubj
ten ADP case
Eersten PROPN obl
ordre NOUN fixed
geliefde ADJ obj
te ADP mark
stellen VERB ccomp
, PUNCT punct
aen ADP case
wie PRON obl
dat SCONJ obj
men PRON nsubj
, PUNCT punct
de DET det
gestipuleerde VERB amod
recognitie NOUN iobj
goederen NOUN nmod

In [5]:
# Create a versions of the sample where each token is masked one by one with <mask> special token for RoBERTa models and [MASK] special token for BERT models

text_per_token = []
doc = nlp(example)
for token in doc:
    text_per_token.append(token.text)

all_masked = []
all_masked_ro = []
for i in range(0, len(text_per_token)):
    new = text_per_token[:i]+['[MASK]']+text_per_token[i+1:]
    new_ro = text_per_token[:i]+['<mask>']+text_per_token[i+1:]
    all_masked.append(new)
    all_masked_ro.append(new_ro)
      

mask_input_pos = []
for item in all_masked:
    mask_input_pos.append(' '.join(item))
    
mask_input_pos_ro = []
for item in all_masked_ro:
    mask_input_pos_ro.append(' '.join(item))

# printing example of the data
print(mask_input_pos[:10])

['[MASK] sahit Inhamet , op ontfangst van dat „ schrijvens , dato 29„e xb : te laten versoecken , dat hij ten Eersten ordre geliefde te stellen , aen wie dat men , de gestipuleerde recognitie goederen nu geeven bal , en niet verpligt , als pro dato soo Lange aen tehouden , sulx thans de zijde buijten belastinge daer van overgaet , en na deesen dat bedragen , eerst het comptoir generael aengereekend ; en ten Lasten gebragt sal kunnen werden', 'Mier [MASK] Inhamet , op ontfangst van dat „ schrijvens , dato 29„e xb : te laten versoecken , dat hij ten Eersten ordre geliefde te stellen , aen wie dat men , de gestipuleerde recognitie goederen nu geeven bal , en niet verpligt , als pro dato soo Lange aen tehouden , sulx thans de zijde buijten belastinge daer van overgaet , en na deesen dat bedragen , eerst het comptoir generael aengereekend ; en ten Lasten gebragt sal kunnen werden', 'Mier sahit [MASK] , op ontfangst van dat „ schrijvens , dato 29„e xb : te laten versoecken , dat hij ten Eers

In [6]:
# Run fill-mask for model over each version of the sample created in the cell above.
# The predicted token will subsequently be manually labeled with a POS-tag label

output_gysbert = []
for inp in mask_input_pos:
    output_gysbert.append(p_gys(inp))


tokens_predicted = []
for item in output_gysbert:
    for d in item[:1]:
        tokens_predicted.append(d['token_str'])

print(len(tokens_predicted))

for token in tokens_predicted:
    print(token)

82
den
##s
om
,
den
ordre
van
den
des
request
van
den
anno
.
,
te
doen
weten
,
dat
men
ten
minsten
hadde
gelieve
te
weeten
,
van
##de
,
is
van
de
oude
onroerende
als
te
te
##de
,
ende
niet
anders
is
deselve
de
##osten
,
lange
op
##gaende
,
als
dat
de
gemeente
ende
,
daer
##toe
is
,
en
ten
wie
te
##men
,
door
ten
geheel
,
is
,
dan
ten
eynde
,
soude
moeten
worden


In [8]:
output_gysbert2 = []
for inp in mask_input_pos:
    output_gysbert2.append(p_gys2(inp))

tokens_predicted = []
for item in output_gysbert2:
    for d in item[:1]:
        tokens_predicted.append(d['token_str'])

print(len(tokens_predicted))

for token in tokens_predicted:
    print(token)

82
coopman
##man
##chan
,
bij
ontfang
van
ue
onderdanig
schrijvens
de
vanden
30
7ber
:
heeft
willen
versoeken
,
dat
hij
ten
eersten
ordre
geliefde
te
stellen
,
op
wien
,
is
,
de
zijde
zijde
,
,
toe
kan
,
en
sig
anders
is
deselve
voor
forma
soo
lange
op
tehouden
,
tot
dat
de
copia
-
,
daer
van
gehouden
,
en
bij
dat
sal
is
voor
aen
het
hooft
generael
belast
,
en
ten
laste
gebragt
sal
moeten
werden


In [ ]:
output_xlm = []
for inp in mask_input_pos_ro:
    output_xlm.append(p_xlm(inp))

tokens_predicted = []
for item in output_xlm:
    for d in item[:1]:
        tokens_predicted.append(d['token_str'])

print(len(tokens_predicted))

for token in tokens_predicted:
    print(token)

In [ ]:
output_rob = []
for inp in mask_input_pos_ro:
    output_rob.append(p_rob(inp))

tokens_predicted = []
for item in output_rob:
    for d in item[:1]:
        tokens_predicted.append(d['token_str'])

print(len(tokens_predicted))

for token in tokens_predicted:
    print(token)

## Part 2: Fill-mask for events

In [ ]:
# Providing versions of the sample where an event trigger is masked with the <mask> special token per sample

examples_masked = ["op <mask> van dat „schrijvens, dato 29„e xb: te laten versoecken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden", "op ontfangst van dat „schrijvens, dato 29„e xb: te laten <mask>, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden", "op ontfangst van dat „schrijvens, dato 29„e xb: te laten verstuiken, dat hij ten Eersten <mask> geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden",  "op ontfangst van dat „schrijvens, dato 29„e xb: te laten versoecken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu <mask> bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden", "op ontfangst van dat „schrijvens, dato 29„e xb: te laten versoecken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet <mask>, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden", "op ontfangst van dat „schrijvens, dato 29„e xb: te laten verstuiken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten <mask> daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden", "op ontfangst van dat „schrijvens, dato 29„e xb: te laten versoecken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael <mask>; en ten Lasten gebragt sal kunnen werden"]

In [35]:
# Providing versions of the sample where an event trigger is masked with the [MASK] special token per sample

examples_masked_bert = ["op [MASK] van dat „schrijvens, dato 29„e xb: te laten versoecken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden", "op ontfangst van dat „schrijvens, dato 29„e xb: te laten [MASK], dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden", "op ontfangst van dat „schrijvens, dato 29„e xb: te laten verstuiken, dat hij ten Eersten [MASK] geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden",  "op ontfangst van dat „schrijvens, dato 29„e xb: te laten versoecken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu [MASK] bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden", "op ontfangst van dat „schrijvens, dato 29„e xb: te laten versoecken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet [MASK], als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden", "op ontfangst van dat „schrijvens, dato 29„e xb: te laten verstuiken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten [MASK] daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael aengereekend; en ten Lasten gebragt sal kunnen werden", "op ontfangst van dat „schrijvens, dato 29„e xb: te laten versoecken, dat hij ten Eersten ordre geliefde te stellen, aen wie dat men, de gestipuleerde recognitie goederen nu geeven bal, en niet verpligt, als pro dato soo Lange aen tehouden, sulx thans de zijde buijten belastinge daer van overgaet, en na deesen dat bedragen, eerst het comptoir generael [MASK]; en ten Lasten gebragt sal kunnen werden"]

In [37]:
# Running relevant fill-mask pipeline (as defined at the beginning of the notebook) over examples

output_robbert = []
for example in examples_masked:
    output_robbert.append(p_rob(example))
    
# Showing output

orig = ['ontfangst', 'versoucken', 'ordre', 'geeven', 'verpligt', 'belastinge', 'aengereekend']
i=-1
for item in output_robbert:
    i+=1
    try:
        print(orig[i])
    except IndexError:
        continue
    for d in item:
        print(d['token_str'], d['score'])
    print()

In [40]:
# Running relevant fill-mask pipeline (as defined at the beginning of the notebook) over examples

output_gysbert = []
for example in examples_masked_bert:
    output_gysbert.append(p_gys(example))
    
# Showing output

orig = ['ontfangst', 'versoucken', 'ordre', 'geeven', 'verpligt', 'belastinge', 'aengereekend']
i=-1
for item in output_gysbert:
    i+=1
    try:
        print(orig[i])
    except IndexError:
        continue
    for d in item:
        print(d['token_str'], d['score'])
    print()

In [42]:
# Running relevant fill-mask pipeline (as defined at the beginning of the notebook) over examples

output_gysbert2 = []
for example in examples_masked_bert:
    output_gysbert2.append(p_gys2(example))
    
# Showing output

orig = ['ontfangst', 'versoucken', 'ordre', 'geeven', 'verpligt', 'belastinge', 'aengereekend']
i=-1
for item in output_gysbert2:
    i+=1
    try:
        print(orig[i])
    except IndexError:
        continue
    for d in item:
        print(d['token_str'], d['score'])
    print()

In [117]:
# Running relevant fill-mask pipeline (as defined at the beginning of the notebook) over examples

output_xlm = []
for it in examples_masked:
    output_xlm.append(p_xlm(it))
    
# Showing output
    
orig = ['ontfangst', 'versoucken', 'ordre', 'geeven', 'verpligt', 'belastinge', 'aengereekend']
i=-1
for item in output_xlm:
    i+=1
    try:
        print(orig[i])
    except IndexError:
        continue
    for d in item:
        print(d['token_str'], d['score'])
    print()

ontfangst
grond 0.5468719601631165
basis 0.09809362888336182
Grund 0.009096396155655384
aanleiding 0.00790109671652317
grond 0.007497806567698717

versoucken
weten 0.7687774896621704
zien 0.04163489118218422
wissen 0.024443605914711952
merken 0.017368324100971222
horen 0.016624892130494118

ordre
de 0.21292002499103546
, 0.06637450307607651
een 0.04304531216621399
is 0.031099610030651093
in 0.023143382743000984

geeven
te 0.041331153362989426
reeds 0.023561103269457817
al 0.022296059876680374
betalen 0.019724750891327858
, 0.017660323530435562

verpligt
meer 0.16891001164913177
langer 0.07015692442655563
, 0.05188818648457527
zoo 0.028032395988702774
zo 0.021120930090546608

belastinge
, 0.2973562180995941
d 0.03696240112185478
s 0.03526788577437401
en 0.03501639515161514
der 0.034940339624881744

aengereekend
, 0.08203158527612686
aan 0.026581211015582085
zal 0.02611595392227173
naar 0.0175927821546793
dient 0.01372816413640976



In [122]:
# Running relevant fill-mask pipeline (as defined at the beginning of the notebook) over examples

output_bertje = []
for it in examples_masked_bert:
    output_bertje.append(p_bertje(it))
    
# Showing output

orig = ['ontfangst', 'versoucken', 'ordre', 'geeven', 'verpligt', 'belastinge', 'aengereekend']
i=-1
for item in output_bertje:
    i+=1
    try:
        print(orig[i])
    except IndexError:
        continue
    for d in item:
        print(d['token_str'], d['score'])
    print()

ontfangst
grond 0.2215891033411026
basis 0.03860171511769295
om 0.013431262224912643
[UNK] 0.012606052681803703
##gaande 0.008121632970869541

versoucken
zien 0.06442654877901077
weten 0.04080161079764366
blijken 0.03742391988635063
verstaan 0.022388309240341187
zeggen 0.018380574882030487

ordre
[UNK] 0.01952861249446869
x 0.011448463425040245
y 0.01137145608663559
ö 0.011317512020468712
##ô 0.011171597987413406

geeven
##t 0.03896583244204521
##n 0.02520104870200157
##s 0.0163403432816267
##r 0.013123624958097935
[UNK] 0.009236494079232216

verpligt
heeft 0.01943816803395748
anders 0.016239339485764503
meer 0.016044940799474716
[UNK] 0.014253392815589905
had 0.012538667768239975

belastinge
##de 0.06359873712062836
##e 0.03373003378510475
##d 0.026918666437268257
van 0.024648962542414665
##t 0.02250092476606369

aengereekend
is 0.09746285527944565
wordt 0.04575735703110695
##t 0.027850385755300522
worden 0.025978608056902885
heeft 0.017592009156942368

